## Subject-Verb-Objects

In this notebook, we conduct a series of experiments in order, first, toisolate the subject-verb-object (SVO) triples in the texts of speakers we have gendered male or female, then to explore the *character spaces* they establish for gendered entities within their speech as well as the nature of the *character space* they create for themselves as speakers. 

**Note**: We are not excluding parentheticals in this notebook.

**Next Steps**: Work on code to compile / visualize this as a network graph (?).

## Load Libraries & Data

In [1]:
# IMPORTS
import re, spacy, textacy
import pandas as pd
from nltk import sent_tokenize

# Loading the Data in a gendered partitioned fashion: 
talks_m = pd.read_csv('talks_male.csv', index_col='Talk_ID')
talks_f = pd.read_csv('talks_female.csv', index_col='Talk_ID')
talks_nog = pd.read_csv('talks_nog.csv', index_col='Talk_ID')
talks_all = pd.concat([talks_m, talks_f, talks_nog])

# And then grabbing on the texts of the talks:
texts_all = talks_all.text.tolist()
texts_women = talks_f.text.tolist()
texts_men = talks_m.text.tolist()

print(f"From our {talks_all.shape[0]}x{talks_all.shape[1]} CSV, \
we have a list of {len(texts_all)} talks: {len(texts_women)} by women and \
{len(texts_men)} by men.")

From our 992x14 CSV, we have a list of 992 talks: 260 by women and 720 by men.


Lowercasing everything upfront because we don't care whether it is *She* or *she*. 

In [2]:
# Lowercase everything before we create spaCy doc and Textacy SVO triple
texts_w = [text.lower() for text in texts_women]
texts_m = [text.lower() for text in texts_men]

spaCy has three different English language models: small, medium, and large. We use the large model here because our corpus is small and the syntax may be a bit more involved. 

`TO DO`: determine the difference between the models. 

After determining telling spaCy which model to use, we then use its conventions for feeding a set of texts as a list of strings, to it. 

The preview simply checks that everything went as planned: it gives us a word count and the first 50 characters -- which is weird because in theory it has converted the string to a series of spacy objects. 

In [3]:
# Load the Space pipeline to be used
nlp = spacy.load('en_core_web_lg')

# Use the pipe method to feed documents 
docs_w = list(nlp.pipe(texts_w))
docs_m = list(nlp.pipe(texts_m))

# A quick check of our work:
docs_m[0]._.preview

'Doc(2690 tokens: "  thank you so much, chris. and it\'s truly a gr...")'

## SVO Dataframes for Male and Female Speakers

We now have two sets of spaCy documents, one for the women speakers and one for the men speakers -- there is a third set, composed of groups of speakes, but we are leaving them aside for now. 

In order to examine how each set of texts imagines the agency and actions of gendered persons in their speech, we are going to go through all the clauses in each talk, identify those clauses that have a pronoun as a subject. We are going to compile all the clauses that feature all the subject case pronouns: *first person* (I and we), *second person* (you), and *third person* (she, he, they, it).

`TO DO`: Identify the clauses where gendered entities are acted upon: her, him, me, us, them. (Convert the **spaCy object** in the function below to strings so that we can then match for the objective pronouns?) This may or may not be possibly best done as a separate section.

In the next two cells we create the list of pronouns for whom we seek SVO triples and then we have the function that will grab SVOs starting with those pronouncs and append them to a list each item of which is a dictionary. Each dictionary consists of three entries: one for the subject, one for the verb, and one for the object. This structure was chosen because it makes it easier to instantiate a pandas dataframe in the work that follows.

In [11]:
# Create lists of gendered words

subjects = 'i she he it you man woman'

objects = 'him her'

gendered = subjects.split(' ')

print(gendered)

['i', 'she', 'he', 'it', 'you', 'man', 'woman']


In [12]:
def svos(doc, svo_list):
    svotriples = list(textacy.extract.triples.subject_verb_object_triples(doc))
    for item in svotriples:
        svo_list.append(
            {
                'subject': str(item[0][-1]), 
                'verb': str(item[1][-1]), 
                'object': str(item[2])
            }
        )

In [14]:
# Create the two lists
svolist_m = []
svolist_w = []

# Populate the lists with SVO triples
for doc in docs_m:
    svos(doc, svolist_m)

for doc in docs_w:
    svos(doc, svolist_w)

# Convert the lists to dataframes
svos_w = pd.DataFrame(svolist_w)
svos_m = pd.DataFrame(svolist_m)

print(df_all_svos_m.shape, df_all_svos_w.shape)

(80331, 3) (26527, 3)


# Counting Verbs

The naming convention for variables here is a little redundant in including `svos` as the prefix for everything, but what follows is which speaker subcorpus being used, `w` for women and `m` for men, and then what subject, `he` or `she` for example, and then in the case of objects, what verb: I would like to discover if there's a way to grab the top 20 verbs and then gather the objects associated with them all at once. (There might be an interesting visualization that could come out of it. Maybe.)

In [25]:
svos_w_he = svos_w[svos_w["subject"] == "he"]
# grp = svos_w_he.groupby("verb")

w_he_verbs = svos_w_he.groupby("verb").size().reset_index(name='obs').sort_values(['obs'], ascending=False)

w_he_verbs.iloc[:20]

,verb,obs
113,had,44
241,said,25
109,going,24
110,got,17
118,has,17
98,found,16
324,wanted,16
304,took,14
120,have,14
58,did,13


In [26]:
svos_w_she = svos_w[svos_w["subject"] == "she"]
# grp = svos_w_he.groupby("verb")

w_she_verbs = svos_w_she.groupby("verb").size().reset_index(name='obs').sort_values(['obs'], ascending=False)

w_she_verbs.iloc[:20]

,verb,obs
112,had,30
114,has,22
229,said,21
278,told,15
105,going,10
71,doing,9
104,goes,9
115,have,9
107,got,8
156,loves,7


# Verbs with Objects

**Needed**: grab the verbs from above as a list and then feed them into the query below and then pipe those results into a single dataframe. (I suspect there is probably a way to write a series of functions that would simply do all this.)

In [28]:
# The df.query() syntax used below if from: 
# https://stackoverflow.com/questions/22086116/how-do-you-filter-pandas-dataframes-by-multiple-columns

svos_w_she_had = svos_w.query('subject == "she" & verb == "had" ')
svos_w_she_had.shape

(30, 3)

In [29]:
svos_w_she_had.head(30)

,subject,verb,object
321,she,had,[baby]
1183,she,had,[money]
1214,she,had,"[vagina, replacement, parts]"
1225,she,had,[jeep]
1716,she,had,[eye]
2989,she,had,[order]
4368,she,had,[day]
5366,she,had,"[to, pull, over, from, the, side, of, the, roa..."
5708,she,had,[problem]
5784,she,had,[rapport]


In [30]:
svos_w_he_had = svos_w.query('subject == "he" & verb == "had" ')
svos_w_he_had.head(30)

,subject,verb,object
1641,he,had,[schooling]
1697,he,had,[belt]
1698,he,had,[thing]
1707,he,had,[her]
3776,he,had,"[chicken, soup]"
4894,he,had,"[lobe, seizures]"
5120,he,had,[concept]
5472,he,had,[injury]
5473,he,had,[scoliosis]
5487,he,had,[body]


## Men speakers on gendered subjects

And now for the men speakers:

In [31]:
svos_m_he_had = svos_m.query('subject == "he" & verb == "had" ')
svos_m_she_had = svos_m.query('subject == "she" & verb == "had" ')
svos_m_he_had.head(10)

,subject,verb,object
405,he,had,[girlfriend]
2658,he,had,[sum]
2819,he,had,[breakthrough]
2845,he,had,[mountain]
2988,he,had,[hair]
4560,he,had,[problems]
4567,he,had,[theory]
4569,he,had,"[grain, data]"
5058,he,had,[same]
5059,he,had,[name]


In [32]:
svos_m_she_had.head(10)

,subject,verb,object
473,she,had,[adhd]
490,she,had,[career]
1230,she,had,[piece]
5527,she,had,[pair]
6131,she,had,"[to, do, it]"
6704,she,had,"[party, something]"
12849,she,had,"[wedding, ring]"
12917,she,had,[memory]
14825,she,had,[test]
17156,she,had,[feeling]
